<a href="https://colab.research.google.com/github/alexgshaw/vol3project2/blob/main/The_Generated_Word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Grant White, Johnson Merrell, Walker Hughes, Alex Shaw

# Introduction

Can we train a computer to write new church hymns? Our project is seeking to answer this question. We will be attempting to recreate both lyrics and music. In this draft we focus on text generation. 

Text generation has become a hot topic in the past decade, and has begun seeing great success in recent years. Applying these methods to song generation will be a new aspect to explore. Music generation has not seen the same amount of success as text generation, but is still an active field of research.

# Data Cleaning

We will use the lyrics from all the Hymns of the Church of Jesus Christ of Latter-day Saints as well as the instrumental tracks. Both of these are available at www.churchofjesuschrist.org. There are 341 hymns available in English.

The first step will be processing our data. This will be done in the following five steps:

1. Split each word into it's own line.
1. Remove verse numbers.
1. Replace sentence-ending punctuation and end of songs with stop words.
1. Remove all remaining punctuation.

The code for data cleaning is given below.

In [ ]:
# Read in our file of lyrics (all songs from the hymn book)
filename = 'lyrics.txt'
with open(filename) as file:
    lyrics = file.read().split()

In [ ]:
# Get rid of all the empty strings
not_empty = lambda x: len(x) != 0
lyrics = list(filter(not_empty, lyrics))

# Replace '1.' with end of song word
end_of_song = 'END'
lyrics = [lyric if lyric != '1.' else end_of_song for lyric in lyrics]
lyrics = lyrics[1:]
lyrics.append(end_of_song)

# Strip out all the verse numbers
not_verse_num = lambda x: not (x[0].isnumeric() and x.endswith('.'))
lyrics = list(filter(not_verse_num, lyrics))

In [ ]:
# Replace all periods with a stop word
period = 'PERIOD'
exclamation = 'EXCLAMATION'
question = 'QUESTION'
semi_colon = 'SEMI'

stop_lyrics = []

for lyric in lyrics:
    if lyric.endswith('.'):
        lyric = lyric.replace('.', '')
        stop_lyrics.append(lyric)
        stop_lyrics.append(period)
    elif lyric.endswith(';'):
        lyric = lyric.replace(';', '')
        stop_lyrics.append(lyric)
        stop_lyrics.append(semi_colon)
    elif lyric.endswith('!'):
        lyric = lyric.replace('!', '')
        stop_lyrics.append(lyric)
        stop_lyrics.append(exclamation)
    elif lyric.endswith('?'):
        lyric = lyric.replace('?', '')
        stop_lyrics.append(lyric)
        stop_lyrics.append(question)
    else:
        stop_lyrics.append(lyric)

lyrics = stop_lyrics

In [ ]:
def process_lyric(lyric):
    """Gets rid of punctuation and non-latin characters."""
    apostrophe = 'â\x80\x99'
    a_hat = 'â'
    lyric = lyric.replace(apostrophe, '')
    lyric = lyric.replace(a_hat, '')
    lyric = ''.join([char for char in lyric if char.isalpha()])

    return lyric

In [ ]:
# Get rid of punctuation and non-latin characters
lyrics = list(map(process_lyric, lyrics))

In [ ]:
# Write the cleaned lyrics
filename = 'clean_lyrics.txt'
with open(filename, 'w') as file:
    for lyric in lyrics:
        lyric += '\n'
        file.write(lyric)